### A new Celertie Term that fits CARMA

<br>**Author(s):** Weixiang Yu
<br>**Last run:** 06-30-2020
<br>**Short description:** This notebook will follow the investigation in [CARMA->Celerite.ipynb](./CARMA->Celerite.ipynb) to write a 'CARMA_term' class to enable $\texttt{Celerite}$ to fit CARMA processes.

## 0. Setup

In [1]:
# import basic packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import os, sys

# see if local stores mpl style, else use from src
try:
    plt.style.use('yu_basic')
except:
    mpl.rc_file('https://raw.githubusercontent.com/ywx649999311/project_template'
                '/master/%7B%7Bcookiecutter.project_name%7D%7D/src/vis/mpl/yu_basic.rc')

pd.set_option('display.max_columns', 999)
%matplotlib inline

In [2]:
import celerite
from celerite import GP, terms
from celerite.solver import get_kernel_value, CARMASolver

## 1. New Celerite Terms (DRW, DHO, CARMA)

In [3]:
class DRW_term(terms.Term):
    """
    Damped Random Walk term with two parameters, 'log_sigma' and 'log_tau'.
    
    Args:
        log_sigma(float): Sigma is the standard deviation of the DRW process.
        log_tau(float): Tau is the characteristic timescale of the DRW process.
    """
    parameter_names = ('log_sigma', 'log_tau')
    
    def get_real_coefficients(self, params):
        log_sigma, log_tau = params
        
        return (
            np.exp(2*log_sigma),
            1/np.exp(log_tau)
        )
    
    def get_perturb_amp(self):
        log_sigma, log_tau = self.get_parameter_vector()
        
        return np.exp((log_sigma-np.log(1/2)-log_tau)/2)
    
    
    def get_rms_amp(self, params):
        log_sigma, log_tau = params
        
        return np.exp(log_sigma)

In [4]:
class DHO_term(terms.Term):
    
    parameter_names = ('log_a1', 'log_a2', 'log_b0', 'log_b1')
    
    def acf(self, arparam, maparam):
        p = len(arparam)
        q = len(maparam)-1    
        sigma = maparam[0]

        # MA param into Kelly's notation
        arparam = np.array(arparam)
        maparam = np.array([x/sigma for x in maparam])

        # get roots
        arroots = np.roots(np.append([1], arparam))
    #     maroots = np.roots(n_maparam[::-1])

        # init acf product terms
        num_left = 0
        num_right = 0
        denom = -2*arroots.real + np.zeros_like(arroots)*1j 

        for k in range(q+1):
            num_left += maparam[k]*np.power(arroots, k)
            num_right += maparam[k]*np.power(-arroots, k)

        for j in range(1, p):
            root_idx = np.arange(p)
            root_k = arroots[np.roll(root_idx, j)]
            denom *= (root_k - arroots)*(np.conj(root_k) + arroots)

        return sigma**2*num_left*num_right/denom

    def get_real_coefficients(self, params):
        log_a1, log_a2, log_b0, log_b1 = params
        
        # params to normal scale
        arpar = np.exp([log_a1, log_a2])
        mapar = np.exp([log_b0, log_b1])
        
        # get roots and acf
        roots = np.roots(np.append([1], arpar))
        acf = self.acf(arpar, mapar)
        
        ar = []
        cr = []
    
        mask = np.iscomplex(roots)
        acf_real = acf[~mask]
        roots_real = roots[~mask]

        for i in range(len(acf_real)):
            ar.append(acf_real[i].real)
            cr.append(-roots_real[i].real)
        return (ar, cr)
    
    def get_complex_coefficients(self, params):
        log_a1, log_a2, log_b0, log_b1 = params
        
        # params to normal scale
        arpar = np.exp([log_a1, log_a2])
        mapar = np.exp([log_b0, log_b1])
        
        # get roots and acf
        roots = np.roots(np.append([1], arpar))
        acf = self.acf(arpar, mapar)
        
        ac = []
        bc = []
        cc = []
        dc = []

        mask = np.iscomplex(roots)
        acf_complex = acf[mask]
        roots_complex = roots[mask]

        for i in range(len(acf_complex)):

            # only take every other root/acf
            if (i%2 == 0):            
                ac.append(2*acf_complex[i].real)
                bc.append(2*acf_complex[i].imag)
                cc.append(-roots_complex[i].real)
                dc.append(-roots_complex[i].imag)            

        return (ac, bc, cc, dc)        

In [5]:
class CARMA_term(terms.Term):

    def __init__(self, log_arpars, log_mapars, *args, **kwargs):
        
        arpar_temp = 'log_a{}'
        mapar_temp = 'log_b{}'
        arpar_names = ('log_a1',)
        mapar_names = ('log_b0',)      
        
        self.arpars = np.exp(log_arpars)
        self.mapars = np.exp(log_mapars)
        self.p = len(self.arpars)
        self.q = len(self.mapars)-1
        
        # combine ar and ma params into one array
        log_pars = np.append(log_arpars, log_mapars)
        
        # loop over par array to find out how many params
        for i in range(2, self.p+1):
            arpar_names += (arpar_temp.format(i),)
            
        for i in range(1, self.q+1):
            mapar_names += (mapar_temp.format(i),)
        
        self.parameter_names = arpar_names + mapar_names
        super(CARMA_term, self).__init__(*log_pars, **kwargs)
        
    def acf(self, arparam, maparam):
        p = len(arparam)
        q = len(maparam)-1    
        sigma = maparam[0]

        # MA param into Kelly's notation
        arparam = np.array(arparam)
        maparam = np.array([x/sigma for x in maparam])

        # get roots
        arroots = np.roots(np.append([1], arparam))
    #     maroots = np.roots(n_maparam[::-1])

        # init acf product terms
        num_left = 0
        num_right = 0
        denom = -2*arroots.real + np.zeros_like(arroots)*1j 

        for k in range(q+1):
            num_left += maparam[k]*np.power(arroots, k)
            num_right += maparam[k]*np.power(-arroots, k)

        for j in range(1, p):
            root_idx = np.arange(p)
            root_k = arroots[np.roll(root_idx, j)]
            denom *= (root_k - arroots)*(np.conj(root_k) + arroots)

        return sigma**2*num_left*num_right/denom

    def get_real_coefficients(self, params):
        
        # get roots and acf
        roots = np.roots(np.append([1], self.arpars))
        acf = self.acf(self.arpars, self.mapars)
        
        ar = []
        cr = []
    
        mask = np.iscomplex(roots)
        acf_real = acf[~mask]
        roots_real = roots[~mask]

        for i in range(len(acf_real)):
            ar.append(acf_real[i].real)
            cr.append(-roots_real[i].real)
        return (ar, cr)
    
    def get_complex_coefficients(self, params):
                
        # get roots and acf
        roots = np.roots(np.append([1], self.arpars))
        acf = self.acf(self.arpars, self.mapars)
        
        ac = []
        bc = []
        cc = []
        dc = []

        mask = np.iscomplex(roots)
        acf_complex = acf[mask]
        roots_complex = roots[mask]

        for i in range(len(acf_complex)):

            # only take every other root/acf
            if (i%2 == 0):            
                ac.append(2*acf_complex[i].real)
                bc.append(2*acf_complex[i].imag)
                cc.append(-roots_complex[i].real)
                dc.append(-roots_complex[i].imag)            

        return (ac, bc, cc, dc)        

## 2. Compare the output of fixed order and more general CARMA terms

### 2.1 CARMA(2,1) Real

In [40]:
dho = DHO_term(np.log(2), np.log(0.8), np.log(1), np.log(0.5))
dho.get_all_coefficients()

[array([-0.0920085,  0.4670085]),
 array([1.4472136, 0.5527864]),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64)]

In [41]:
carma = CARMA_term(np.log([2,0.8]), np.log([1,0.5]))
carma.get_all_coefficients()

[array([-0.0920085,  0.4670085]),
 array([1.4472136, 0.5527864]),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64)]

### 2.2 CARMA(2,1) Complex

In [42]:
dho = DHO_term(np.log(2), np.log(1.2), np.log(1), np.log(3))
dho.get_all_coefficients()

[array([], dtype=float64),
 array([], dtype=float64),
 array([2.45833333]),
 array([4.56530545]),
 array([1.]),
 array([-0.4472136])]

In [44]:
carma = CARMA_term(np.log([2,1.2]), np.log([1,3]))
carma.get_all_coefficients()

[array([], dtype=float64),
 array([], dtype=float64),
 array([2.45833333]),
 array([4.56530545]),
 array([1.]),
 array([-0.4472136])]

### 2.3 CARMA(1,0)
Note that `DRW_term` takes $\tau$ and $\sigma$ as the standard deviation

In [6]:
carma = CARMA_term(np.log([1/200]), np.log([0.1]))
carma.get_all_coefficients()

[array([1.]),
 array([0.005]),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64)]

In [7]:
drw = DRW_term(np.log(1), np.log(200))
drw.get_all_coefficients()

[array([1.]),
 array([0.005]),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64)]